<a href="https://colab.research.google.com/github/iceshadow2107/AIDEA-Text-labeling-and-identification-of-agricultural-articles/blob/main/ALL_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import monpa
from monpa import utils
monpa.load_userdict("./userdict.txt") # 匯入字典
import os
import csv

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


# 修改成第一階段以及private資料

只改檔案名即可(first_data是有label的檔案，final_data是Private測試集階段)(因為寫test跟train我會搞混......)

In [ ]:
def monpa_function(filenames):
    process_data = {}
    for file in filenames:
        f = open(filepath + file, mode='r', encoding="utf-8")
        text = f.read()
        idx = os.path.splitext(file)[0]
        if idx not in process_data.keys():
            process_data[idx] = text

    for key in list(process_data.keys()):
        result = []
        for item in utils.short_sentence(process_data[key]):
            if item:
                result.append(monpa.cut(item))
        strA = ' '.join('%s' %id for id in result)
        process_data[key] = strA
    return process_data

In [ ]:
%%time
filepath = './dataTrainComplete/'
first_data = monpa_function(os.listdir(r'./dataTrainComplete'))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/monpa/crf_layer.py:374: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


CPU times: user 1min 52s, sys: 37.9 s, total: 2min 30s
Wall time: 1min 56s


In [ ]:
%%time
filepath = './dataPrivateComplete/'
final_data = monpa_function(os.listdir(r'./dataPrivateComplete'))

CPU times: user 1min 20s, sys: 27.3 s, total: 1min 47s
Wall time: 1min 23s


# 修改成第一階段以及private資料你想要輸出的檔名

只改輸出的檔案名即可(first_data是有label的檔案，final_data是Private測試集階段)(因為寫test跟train我會搞混......)

In [ ]:
with open('first_data_monpa.csv', mode='w', encoding="utf-8") as f:  
    writer = csv.writer(f)
    for k, v in first_data.items():
        writer.writerow([k, v])

In [ ]:
with open('final_data_monpa.csv', mode='w', encoding="utf-8") as f:  
    writer = csv.writer(f)
    for k, v in final_data.items():
        writer.writerow([k, v])

# 製作第一階段的Train label sample

In [ ]:
first_data = pd.read_csv('first_data_monpa.csv', header=None, index_col=0, squeeze=True).to_dict()

In [ ]:
TrainLabel = pd.read_csv('TrainLabel.csv', header=0, encoding='utf-8')
TrainLabel_test = TrainLabel['Test'].values.tolist()
TrainLabel_Reference = TrainLabel['Reference'].values.tolist()

In [ ]:
TrainLabel_test_text = []
TrainLabel_Reference_text = []
for item in TrainLabel_test:
    TrainLabel_test_text.append(str(first_data[item]))
for item in TrainLabel_Reference:
    TrainLabel_Reference_text.append(str(first_data[item]))

In [ ]:
pn = [1]*len(TrainLabel)

In [ ]:
TrainLabel_text_1 = pd.DataFrame({'Test':TrainLabel_test, 'Reference':TrainLabel_Reference, 'Test_text':TrainLabel_test_text, 'Reference_text':TrainLabel_Reference_text, 'P/N':pn})

In [ ]:
list_key_Label = list(first_data.keys())
LabellistA = [val for val in list_key_Label for i in range(len(list_key_Label))]
LabellistB = list_key_Label*len(list_key_Label)

In [ ]:
TrainLabel_test_text = []
TrainLabel_Reference_text = []
for item in LabellistA:
    TrainLabel_test_text.append(str(first_data[item]))
for item in LabellistB:
    TrainLabel_Reference_text.append(str(first_data[item]))

In [ ]:
d = [str(list(first_data.keys())[i])*2 for i in range(len(first_data.keys()))]
LabellistC = list(str(LabellistA[i])+str(LabellistB[i]) for i in range(len(LabellistA)))
TrainLabel_combo = list((str(TrainLabel_test[i])+str(TrainLabel_Reference[i]) for i in range(len(TrainLabel)))) + d

In [ ]:
PN = []
for i in range(len(LabellistC)):
    if LabellistC[i] in TrainLabel_combo:
        PN.append(1)
    else:
        PN.append(0)

In [ ]:
TrainLabel_text_all = pd.DataFrame({'Test':LabellistA, 'Reference':LabellistB, 'Test_text':TrainLabel_test_text, 'Reference_text':TrainLabel_Reference_text, 'P/N':PN})

## 可以改隨機抽取的label 0個數跟模式

In [ ]:
#0的df
TrainLabel_text_0 = TrainLabel_text_all[TrainLabel_text_all['P/N'] == 0]
TrainLabel_text_0_sample = TrainLabel_text_0.sample(n=6500,random_state=123,axis=0)

In [ ]:
TrainLabel_text_sample = pd.concat([TrainLabel_text_1, TrainLabel_text_0_sample], ignore_index=True)

In [ ]:
TrainLabel_text_sample

,Test,Reference,Test_text,Reference_text,P/N
0,3,415,"['夏季', '為', '斜紋', '夜盜', '及', '甜菜夜蛾', '發生', '盛期...","['夜蛾', '拉', '警報', '，', '白粉', '等', '著', '來', '，...",1
1,3,649,"['夏季', '為', '斜紋', '夜盜', '及', '甜菜夜蛾', '發生', '盛期...","['斜紋夜盜蟲', '密度', '升高', '，', '為', '確保', '作物', '收...",1
2,9,5,"['請', '加強', '水稻', '飛蝨類', '害蟲', '防治', '工作', '。'...","['花蓮', '地區', '水稻', '褐飛蝨', '、', '白背飛蝨', '及', '斑...",1
3,25,32,"['發佈', '草莓', '苗期', '炭疽病', '防治', '警報', '，', '苗栗...","['苗改場', '表示', '草莓', '苗期', '炭疽病', '疫情', '上升', '...",1
4,25,41,"['發佈', '草莓', '苗期', '炭疽病', '防治', '警報', '，', '苗栗...","['近來', '氣候', '高溫', '多濕', '，', '易', '發生', '草莓',...",1
...,...,...,...,...,...
7878,588,1130,"['氣候', '不定', '，', '鋒面', '過後', '，', '水稻', '葉', ...","['近來', '天氣', '變化', '大', '，', '花蓮區', '農改場', '提醒...",0
7879,424,1119,"['臺南區', '農改場', '籲請', '農友', '加強', '防治', '水稻', '...","['台中區', '農業', '改良場', '籲請', '農民', '掌握', '時機', '...",0
7880,653,267,"['時值', '檬果', '開', '花期', '，', '台南區', '農業', '改良場...","['梅', '雨', '鋒面', '來襲', '，', '請', '葡萄', '果農', '...",0
7881,230,380,"['水稻', '白葉枯病', '及', '飛蝨類', '現蹤', '，', '且', '正值...","['氣候', '變化', '大', '，', '籲請', '農友', '注意', '防範',...",0


## 可以改成你想要的label檔名

In [ ]:
TrainLabel_text_sample.to_csv('TrainLabel_text_sample.csv', header=True, index=False)

# 製作private train data

In [ ]:
#final_data = pd.read_csv('final_data_monpa.csv', header=None, index_col=0, squeeze=True).to_dict()

In [ ]:
list_key = list(final_data.keys())
listA = [val for val in list_key for i in range(len(final_data))]
listB = list_key*(len(final_data))

In [ ]:
final_data_test_text = []
final_data_Reference_text = []
for item in listA:
    final_data_test_text.append(str(final_data[item]))
for item in listB:
    final_data_Reference_text.append(str(final_data[item]))

In [ ]:
final_data_all = pd.DataFrame({'Test':listA, 'Reference':listB, 'Test_text':final_data_test_text, 'Reference_text':final_data_Reference_text})

## 可以改隨機抽取的private data個數跟模式

In [ ]:
final_data_all.to_csv('Test_data_sample.csv', header=True, index=False)

In [ ]:
final_data_all

,Test,Reference,Test_text,Reference_text
0,1047,1047,"['近來', '天氣', '變化', '大', '，', '苗栗區', '農改場', '籲請...","['近來', '天氣', '變化', '大', '，', '苗栗區', '農改場', '籲請..."
1,1047,1090,"['近來', '天氣', '變化', '大', '，', '苗栗區', '農改場', '籲請...","['水稻', '陸續', '進入', '抽穗期', '，', '花蓮場', '籲請', '農..."
2,1047,1279,"['近來', '天氣', '變化', '大', '，', '苗栗區', '農改場', '籲請...","['適逢', '香蕉', '花', '薊馬', '危害', '發生期', '，', '香蕉'..."
3,1047,1286,"['近來', '天氣', '變化', '大', '，', '苗栗區', '農改場', '籲請...","['香蕉', '所', '籲請', '各', '單位', '宣導', '蕉農', '儘速',..."
4,1047,15,"['近來', '天氣', '變化', '大', '，', '苗栗區', '農改場', '籲請...","['瘤野螟', '與', '飛蝨', '現蹤', '，', '水稻', '二', '期', ..."
...,...,...,...,...
176395,284,1260,"['台東', '農改場', '表示', '近日', '受', '鋒面', '影響', '連日...","['紅龍果', '潰瘍病', '防治', '黃金期', '，', '籲請', '農友', '..."
176396,284,1248,"['台東', '農改場', '表示', '近日', '受', '鋒面', '影響', '連日...","['籲請', '香蕉', '組織', '培養', '苗', '種植', '者', '，', ..."
176397,284,253,"['台東', '農改場', '表示', '近日', '受', '鋒面', '影響', '連日...","['梨', '赤星病', '的', '感染源', '已', '零星', '出現', '，',..."
176398,284,509,"['台東', '農改場', '表示', '近日', '受', '鋒面', '影響', '連日...","['苗栗', '地區', '秋', '行軍蟲', '成', '蟲數', '增加', '，',..."


# 處理訓練集及測試集的stop word 及標點符號 #

In [ ]:
%%time
def clean_seg_list(seg_list):
    punctuations = ['+', '?', '!', '！', '，', '.', '、', ':', '：', ';', '；', ' ', '\/', '_', ',', '$', '^', 
                    '*', '~', '\""', '\\', '/', '／', '「', '」', '...', '......', '#', '(',')', '「','(',' ',')','」',
                    '『', '』', '“', '”', '「', '」', '《', '》', '-', '－', '=', '⋯', '～', '・', '·', '<', '>', '．', '%', 
                    '％']
    stop_words = [line.strip() for line in open ('stopword_chinese.txt', encoding = 'utf-8').readlines()]
    return [seg for seg in seg_list if seg[0] not in punctuations + stop_words]

def get_cleaned_seg(df):
    df['All'] = df['Test_text'] + ' ' + df['Reference_text']
    
    cleaned = []
    for line in df['All']:
        items = line.split()
        item = ' '.join(clean_seg_list(items))
        cleaned.append(item)
    df['All_text'] = cleaned
    df = df.drop('All', axis=1)

    return df
df_train = get_cleaned_seg(TrainLabel_text_sample)
df_test = get_cleaned_seg(final_data_all)

CPU times: user 22min, sys: 4.45 s, total: 22min 4s
Wall time: 22min 6s


# 訓練模型 #

In [ ]:
labels = df_train['P/N'].tolist()
features = df_train['All_text'].tolist()
features_pre = df_test['All_text'].tolist()

In [ ]:
tfidf_BOW = TfidfVectorizer()
tfidf_BOW.fit(features)
kfold = KFold(n_splits = 10,shuffle = True)

In [ ]:
%%time
#RF

model = RandomForestClassifier()
expected = []
predicted_class = []
for train_index, test_index in kfold.split(features):
    x_train, x_test = np.array(features)[train_index], np.array(features)[test_index]
    y_train, y_test = np.array(labels)[train_index], np.array(labels)[test_index]

    vectors_train = tfidf_BOW.transform(x_train)
    vectors_test = tfidf_BOW.transform(x_test)

    model.fit(vectors_train, y_train)

    expected.extend(y_test)
    predicted_class.extend(model.predict(vectors_test))

print ("Classification report for classifier: \n %s" % (metrics.classification_report(expected, predicted_class)))
print ("Confusion matrix: \n%s" % metrics.confusion_matrix(expected, predicted_class))

Classification report for classifier: 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      6500
           1       0.92      0.78      0.85      1383

    accuracy                           0.95      7883
   macro avg       0.94      0.88      0.91      7883
weighted avg       0.95      0.95      0.95      7883

Confusion matrix: 
[[6410   90]
 [ 305 1078]]
CPU times: user 1min 9s, sys: 1.35 s, total: 1min 10s
Wall time: 1min 11s


# 預測模型 #

In [ ]:
%%time
x_pre = np.array(features_pre)
vectors_pre = tfidf_BOW.transform(x_pre)

class_pre = []
class_pre.extend(model.predict(vectors_pre))

CPU times: user 42.2 s, sys: 2.22 s, total: 44.4 s
Wall time: 46.7 s


# 存檔 #

In [ ]:
df_ans = pd.DataFrame({'ans':class_pre})
df_concat = pd.concat([df_test['Test'],df_test['Reference'],df_ans],axis = 1)

df_final = df_concat[df_concat['ans']==1]
df_submission = df_final.drop('ans',axis = 1)
df_submission.head()

,Test,Reference
28,1047,842
47,1047,502
98,1047,62
114,1047,837
277,1047,804


In [ ]:
df_submission.to_csv('submission_example.csv',header =True,index=False)